In [1]:
def get_skycamera_predictions_apelsvoll():
    import pandas as pd

    import numpy as np
    import cv2
    import os
    import time as t

    import datetime
    import pvlib
    from pvlib.location import Location
    from matplotlib import pyplot as plt

    import mysql.connector # lastes ned ved 'pip install mysql' i command promt
    from load_snowdata import load_snowdata_apelsvoll
    import sys

    sys.path.append(r'C:\Users\heinenr\Documents\git_local\twin_tools\\')

    from twin_tools import get_statistical_twin, remove_outliers, sort_split_dfs

    import pickle

    def get_cloud_map(frame, method='brd_ref'):
        """Takes an RGB-image (numpy-array) and transforms to a cloud map through a image algorithm and naive thresholding"""

        if method=='brd_ref':
            r_ref = np.mean(frame[:, :, 2]) # defining mean red value for polar converted image
            b_ref = np.mean(frame[:, :, 0]) # defining mean blue value for polar converted image

            cloud_map = np.subtract((frame[:, :, 0]-b_ref), (frame[:, :, 2]-r_ref)).astype(np.uint8)           

            cloud_map[cloud_map<(np.mean(cloud_map))]=0
            cloud_map[cloud_map>(np.mean(cloud_map))]=255
        else:
            frame=None
            print('Model not recognized. Valid models are brd_ref')
        return cloud_map

    def corners_inside_sun(speed_ave, angle_ave, good_corners, cloud_map, location, prediction_horizon, resize_dim, frames_per_min=2):    
        inside = []


        predicted_point = np.zeros_like(good[-1])

        num_frames=prediction_horizon*frames_per_min #how many frames into the future to predict
        displacement_x = num_frames*speed_ave*np.cos(angle_ave)
        displacement_y = num_frames*speed_ave*np.sin(angle_ave)
        predicted_point[:,0], predicted_point[:,1] = (good_corners[-1][:,0] + displacement_x), (good_corners[-1][:,1] + displacement_y)

        prediction_time = time + pd.Timedelta(prediction_horizon, unit='min')

        #get the solar position in the location and times specified
        solpos = location.get_solarposition(prediction_time)
        rho = resize_dim/2

        # cylindrical
        r = rho*np.sin(np.radians(solpos['apparent_zenith']))
        azimuth = np.radians(solpos['azimuth']) + (9*np.pi/16) # camera is rotated approx 10 degs, so a correction factor of pi/16 is needed (specific for apelsvoll). pi/2 is added to rotate image so that 0 is north. 

        #fisheye distortion
        r_norm = r/rho

        r_dist_norm = 2*(r_norm + (1-np.sqrt(1 -r_norm**2))/2)/3 #must divide the normal fisheye formula by 1.5. Don't know why
        r_dist = r_dist_norm*rho

        # to cartesian (and adjusting to fit picture indices)
        x_dist=r_dist*np.cos(azimuth)+rho
        y_dist=rho-r_dist*np.sin(azimuth)

        points_inside = (predicted_point[:,0]>(x_dist[0]-20))&(predicted_point[:,0]<(x_dist[0]+20))&(predicted_point[:,1]>(y_dist[0]-20))&(predicted_point[:,1]<(y_dist[0]+20))
        points_inside_sum = points_inside.sum()
        pixels = good_corners[-1][points_inside]
        roi_sum = 0

        if len(pixels)>0:
            for pixel in pixels:
                roi = cloud_map[(int(pixel[0])-20):(int(pixel[0])+20), (int(pixel[1])-20):(int(pixel[1])+20)]
                roi_mean = np.mean(roi)
                roi_sum += roi_mean
                roi_ave = roi_sum/len(pixels)
        else:
            roi_ave=0           

        inside.append([time, points_inside_sum, roi_ave])
        columns = [[prediction_horizon], ['num_corners', 'ave_pixel_intensity']]


        inside_df = pd.DataFrame(inside)
        inside_df.index = inside_df.pop(0)
        inside_df.columns = pd.MultiIndex.from_product(columns, names=['time_ahead', 'predictions'])

        return inside_df, predicted_point, x_dist, y_dist

    def get_clearsky_irr(times, location):

        # Run functions
        lin_turb = pvlib.clearsky.lookup_linke_turbidity(times,
                                                         location.latitude,
                                                         location.longitude)

        cSky = location.get_clearsky(times, model='ineichen',
                                    linke_turbidity=lin_turb)
        solarpos = pvlib.solarposition.spa_python(times, location.latitude,
                                                  location.longitude,
                                                  altitude=location.altitude)

        extr_terr_irr = pvlib.irradiance.get_extra_radiation(times)
        AM = location.get_airmass(solar_position=solarpos)
        POA_total = pvlib.irradiance.get_total_irradiance(syst_angles['tilt'],
                                                          syst_angles['azimuth'],
                                                          solarpos['apparent_zenith'],
                                                          solarpos['azimuth'],
                                                          cSky.dni, cSky.ghi, cSky.dhi,
                                                          extr_terr_irr,
                                                          AM.airmass_absolute,
                                                          model='perez')

        POA_total.index = range(1,times.shape[0]+1)
        poa_trans = POA_total.transpose()
        poa_trans.columns = pd.MultiIndex.from_product([list(POA_total.index),['clear_sky_POA']], names=['time_ahead', 'predictions'])
        poa_trans_global = poa_trans.loc['poa_global']
        poa_trans_global.name = time
        poa_trans.index = [time, 'poa_direct','poa_diffuse', 'poa_sky_diffuse','poa_ground_diffuse'] # poa_global is given time-index so that it joins on the correect index with prediction_df
        return poa_trans, POA_total

    directory = r"C:\Users\heinenr\OneDrive - Institutt for Energiteknikk\Prosjekt\Solar Farm\kamerabileter\3_days_july_2019\all_days"
    first_image_path = os.path.join(directory, os.listdir(directory)[-4])
    first = cv2.imread(first_image_path, 1)

    resize_dim = 600
    max_dim = max(first.shape)
    scale = resize_dim/max_dim
    old_frame = cv2.resize(first, None, fx=scale, fy=scale)

    black = np.zeros_like(old_frame)
    circle = cv2.circle(black, (int(black.shape[0]/2), int(black.shape[1]/2)), int(black.shape[0]/2)-25, color=(255,255,255), thickness=-1)
    white_mask = (cv2.cvtColor(circle, cv2.COLOR_BGR2GRAY)/255).astype(np.uint8)

    old_cloud_map = get_cloud_map(old_frame)

    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 10000,
                           qualityLevel = 0.05,
                           minDistance = 1,
                           blockSize = 15 )

    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15,15),
                      maxLevel = 5,
                      criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Create some random colors
    color = np.random.randint(0,255,(10000,3))

    # Take first frame and find corners in it

    p_first = cv2.goodFeaturesToTrack(old_cloud_map, mask = white_mask, **feature_params)

    # Create a mask image for drawing purposes

    num_frames_integrated = 4
    frames_per_min = 2

    syst_angles = {'tilt': 46, 'azimuth': 173}
    location = Location(60.7008333, 10.867777777777778, tz='Europe/Oslo',
                       altitude=273, name='Apelsvoll')
    # found from latlong.net, Østre Toten målestasjon i

    all_predictions = pd.DataFrame()
    evaluated_times = []

    if os.listdir(directory)[-1][-9:-7] == '30':
        half_minute_correction = 9
    else:
        half_minute_correction = 8

    for image in range((len(os.listdir(directory))-half_minute_correction), len(os.listdir(directory))):
        #start = t.time()
        for root, dirs, files in os.walk(directory):
            filenames = []

            for name in files[image:image+num_frames_integrated]:
                filename = os.path.join(root, name)
                filenames.append(filename)

        if len(filenames)<4:
            break

        else:
            p = []
            good = []
            speed = []
            angle = []

            f0=filenames[0]
            mask = np.zeros_like(old_frame)


            frame0 = cv2.imread(f0, 1)
            frame0 = cv2.resize(frame0, None, fx=scale, fy=scale)

            cloud_map0 = get_cloud_map(frame0)

            p.append(cv2.goodFeaturesToTrack(cloud_map0, mask = white_mask, **feature_params))

            if p[0] is None:
                p_placeholder, st0, err = cv2.calcOpticalFlowPyrLK(old_cloud_map, cloud_map0, p_first, None, **lk_params)
                p.append(p_placeholder)
                # Select good points
                good.append(np.ones_like(p_placeholder[st0==1]))
            else:
                p[0] = p[0].astype(np.float32)

                # calculate optical flow
                p_placeholder, st0, err = cv2.calcOpticalFlowPyrLK(old_cloud_map, cloud_map0, p[0], None, **lk_params)
                p.append(p_placeholder)
                good.append(p[0][st0==1])

            good.append(p[1][st0==1])

            speed.append(np.sqrt((good[1][:,1]-good[0][:,1])**2+(good[1][:,0]-good[0][:,0])**2))
            angle.append(np.arctan((good[1][:,1]-good[0][:,1])/(good[1][:,0]-good[0][:,0])))

            old_cloud_map = cloud_map0.copy()

            for frame_num in range(2,(len(filenames))):

                file=filenames[frame_num]

                frame = cv2.imread(file, 1)
                frame = cv2.resize(frame, None, fx=scale, fy=scale)

                cloud_map = get_cloud_map(frame)

                # calculate optical flow
                p_placeholder, st, err = cv2.calcOpticalFlowPyrLK(cloud_map0, cloud_map, p[frame_num-1], None, **lk_params)
                p.append(p_placeholder)
                good.append(p[frame_num][st0==1])

                speed.append(np.sqrt((good[frame_num][:,1]-good[frame_num-1][:,1])**2+(good[frame_num][:,0]-good[frame_num-1][:,0])**2))
                angle.append(np.arctan((good[frame_num][:,1]-good[frame_num-1][:,1])/(good[frame_num][:,0]-good[frame_num-1][:,0])))

                cloud_map0 = cloud_map.copy()

            speed_ave = np.zeros_like(speed[0])
            angle_ave = np.zeros_like(angle[0])

            for i in range(len(speed)):
                speed_sum = np.add(speed_ave,speed[i])
                angle_sum = np.add(angle_ave,angle[i])

            speed_ave = speed_sum/len(speed)
            angle_ave = angle_sum/len(speed)

            time = pd.to_datetime(file[-21:-7])
            evaluated_times.append(time)
            prediction_df = pd.DataFrame()
            for prediction_horizon in range(1,31):

                inside_df, predicted_point, x_dist, y_dist = corners_inside_sun(speed_ave, angle_ave, good, cloud_map, location, prediction_horizon, resize_dim, frames_per_min=2)

                prediction_df = pd.concat([prediction_df, inside_df], axis=1)

            times = pd.date_range(time+pd.Timedelta(1,unit='min'), periods=30, freq='T')

            poa_trans, POA_total = get_clearsky_irr(times, location)
            prediction_df = prediction_df.join(poa_trans).sort_index(axis=1)

            # Now update the previous frame and previous points
            all_predictions = pd.concat([all_predictions, prediction_df], axis=0)
            p[0] = good[1].reshape(-1,1,2)
        #stop = t.time()
        #duration=stop-start
        #print(duration)

    cloud_number_sum = all_predictions.xs('num_corners', level='predictions', axis=1)*all_predictions.xs('ave_pixel_intensity', level='predictions', axis=1)
    normalization_factor = np.array([52702.95375, 51618.52668103, 50534.09961207, 49449.6725431,
           48365.24547414, 47280.81840517, 46196.39133621, 45111.96426724,
           44027.53719828, 42943.11012931, 41858.68306034, 40774.25599138,
           39689.82892241, 38605.40185345, 37520.97478448, 36436.54771552,
           35352.12064655, 34267.69357759, 33183.26650862, 32098.83943966,
           31014.41237069, 29929.98530172, 28845.55823276, 27761.13116379,
           26676.70409483, 25592.27702586, 24507.8499569, 23423.42288793,
           22338.99581897, 21254.56875])
    cloud_number_norm = 1-(cloud_number_sum/normalization_factor)
    cloud_number_norm.columns = pd.MultiIndex.from_product([list(cloud_number_norm.columns), ['weighted_cloud_number']], names=['time_ahead', 'predictions'])
    all_predictions = all_predictions.join(cloud_number_norm).sort_index(axis=1)

    predicted_irr = (all_predictions.xs('weighted_cloud_number', level='predictions', axis=1))*all_predictions.xs('clear_sky_POA', level='predictions', axis=1)
    predicted_irr.columns = pd.MultiIndex.from_product([list(predicted_irr.columns), ['predicted_irradiance']], names=['time_ahead', 'predictions'])
    all_predictions = all_predictions.join(predicted_irr).sort_index(axis=1)
    all_predictions.index.name = 'date_time'

    if pd.Timestamp.now()-pd.Timestamp(all_predictions.index[-1])>pd.Timedelta(1, 'T'):
        print('Skycamera not up to date')

    _user='ifePVsystem'      # brukernavn
    _password='Sol1Sinnet'   # passord
    _host='128.39.229.38'    # evt. kj-sol-01, dette er adressen der serveren bor, og kan bare nåes på IFEs internnett (kabel eller VPN)
    _database='pvs'      # navnet på databasen vår. En host kan ha flere databaser (som en blokk kan ha flere leiligheter)

    query1 = '''SELECT date_time, irradiance_in_plane, irradiance_horizontal, ambient_temperature, panel_t1, panel_t2, panel_t3, panel_t4
                FROM temp_data
                WHERE id_site = 4
                AND (date_time BETWEEN '{0}' AND '{1}') 
                ORDER by date_time'''.format(evaluated_times[-1]-pd.Timedelta(10, unit='T'), evaluated_times[-1])

    connection = mysql.connector.connect(user=_user, password=_password, host=_host, database=_database)
    temp_data = pd.read_sql(query1, con=connection)
    connection.close()

    query2 = '''SELECT date_time, wind_min, wind_max, wind_mid  
                FROM wind_data
                WHERE id_site = 4
                AND (date_time BETWEEN '{0}' AND '{1}') 
                ORDER by date_time'''.format(evaluated_times[-1]-pd.Timedelta(10, unit='T'), evaluated_times[-1])

    connection = mysql.connector.connect(user=_user, password=_password, host=_host, database=_database)
    wind_data = pd.read_sql(query2, con=connection)
    connection.close()

    query3 = '''SELECT date_time, ac_power 
                FROM inverter_data
                WHERE id_site = 4
                AND (date_time BETWEEN '{0}' AND '{1}') 
                ORDER by date_time'''.format(evaluated_times[-1]-pd.Timedelta(10, unit='T'), evaluated_times[-1])

    connection = mysql.connector.connect(user=_user, password=_password, host=_host, database=_database)
    inverter_data = pd.read_sql(query3, con=connection)
    connection.close()

    df_inverter = inverter_data.merge(temp_data).merge(wind_data)
    df_inverter.index = df_inverter.date_time
    df_inverter.drop('date_time', axis=1, inplace=True)
    df_inverter.head()

    snow_index = df_inverter.index.tz_localize('UTC').tz_convert('Europe/Oslo')
    snow = load_snowdata_apelsvoll(str(snow_index[0].date()), str(snow_index[-1].date()+pd.Timedelta(1,unit='D'))).resample(pd.infer_freq(snow_index)).pad()[str(snow_index[0]):str(snow_index[-1])]
    snow = snow[~snow.index.duplicated()]
    snow.index = snow.index.tz_convert('UTC').tz_localize(None)
    df_inverter = df_inverter.join(snow, on=df_inverter.index)
    df_inverter = df_inverter[df_inverter['snow_depth']==0]
    df_inverter.drop(['swe', 'snow_depth'], axis=1, inplace=True)

    saveFolderName = r"C:\Users\heinenr\OneDrive - Institutt for Energiteknikk\Jupyter Notebooks\Prosjekt\Solar Farm\\"

    pickleName = 'apelsvoll_skycamera_predictions_regressors.sav'
    predictions = []
    with open(saveFolderName+pickleName,"rb") as pickle_in:
            regressor_dict = pickle.load(pickle_in)

    for prediction_horizon in range(1,31):
        df_c_i = df_inverter.merge(all_predictions[prediction_horizon], how = 'inner', right_on=['date_time'], left_index=True)

        x = df_c_i.copy()
        x = x[~x.index.duplicated()]

        y = regressor_dict[prediction_horizon].predict(x)
        predictions.append(y[-1])

    predict_df = pd.DataFrame(predictions).transpose()
    predict_df.index = [df_c_i.index[-1]]
    predict_df.index.name = 'date_time'
    predict_df.columns = pd.MultiIndex.from_product([range(1,31)], names=['prediction_horizon (min)'])
    
    return predict_df

predict_df = get_skycamera_predictions_apelsvoll()
print(predict_df)

Skycamera not up to date


C:\Users\heinenr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\heinenr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\heinenr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator DecisionTr

prediction_horizon (min)   1    2    3    4    5    6    7    8    9    10  \
date_time                                                                    
2019-07-16 21:29:00       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

prediction_horizon (min)  ...   21   22   23   24   25        26   27   28  \
date_time                 ...                                                
2019-07-16 21:29:00       ...  0.0  0.0  0.0  0.0  0.0  0.150538  0.0  0.0   

prediction_horizon (min)       29   30  
date_time                               
2019-07-16 21:29:00       2.66622  0.0  

[1 rows x 30 columns]
